# Подготовка

In [16]:
# Java
!apt-get update # Обновляем список пакетов в системе

!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Устанавливаем OpenJDK 8 (Java Development Kit) без графического интерфейса
# > /dev/null: Перенаправляет вывод команды в /dev/null (пустое устройство), чтобы скрыть вывод в консоли

# Spark
!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Загружаем архив Spark 3.1.1, предварительно скомпилированный для Hadoop 3.2, с сайта Apache
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Распаковываем архив

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
--2025-03-28 06:33:45--  https://archive.apache.org/dist/spark/spark-3

In [17]:
# настраеваем переменные окружения
import os  # Импортирует модуль для работы с операционной системой

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"  # Устанавливает переменную окружения JAVA_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2" # Устанавливает переменную окружения SPARK_HOME

In [18]:
# Устанавливаем findspark
!pip install -q findspark

import findspark
findspark.init() # Инициализируем findspark

In [19]:

# Создание SparkContext
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col, max, sum, countDistinct
# Создаем SparkSession, который является точкой входа для работы с DataFrames и SQL в Spark
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

sc = spark.sparkContext  # Получаем SparkContext из SparkSession, SparkContext - это точка входа в Spark

print(sc.version)  # Выводим версию Spark

3.1.1


In [20]:

spark = SparkSession.builder.appName("MySparkApp").getOrCreate() # Создаем SparkSession
# Читаем файлы CSV
# header=True, чтобы использовать заголовки столбцов
trips_df = spark.read.csv("trips.csv", header=True, inferSchema=True)
stations_df = spark.read.csv("stations.csv", header=True, inferSchema=True)

# Отображаем по 2 строки из каждого файла
trips_df.show(n=2)
stations_df.show(n=2)

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|           null|South Van Ness at...|              66|8/29/2013 14:14|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|    null|8/29/2013 14:42|  San Jose City Hall|              10|8/29/2013 14:43|  San Jose City Hall|            10|    661|       Subscriber|   95138|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
only showing top 2 rows

+---+--------------------+-------

In [21]:
# выводим схему данных датафрейма spark соответственно (имена столбцов, типы данных и информация о том, допускает ли столбец значения NULL)
print("Trips")
trips_df.printSchema()

print("Stations")
stations_df.printSchema()

Trips
root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

Stations
root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: string (nullable = true)



# Основное задание

**Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):**

1.   Найти велосипед с максимальным временем пробега.
2.   Найти наибольшее геодезическое расстояние между станциями.
3.   Найти путь велосипеда с максимальным временем пробега через станции.
4.   Найти количество велосипедов в системе.
5.   Найти пользователей потративших на поездки более 3 часов.

---



1.   Найти велосипед с максимальным временем пробега.

In [22]:
# Ищем максимальное время пробега (duration)
max_duration = trips_df.agg(max("duration")).collect()[0][0]

# Ищем велосипед (bike_id) с этим максимальным временем пробега
bike_with_max_duration_df = trips_df.filter(trips_df.duration == max_duration)

# Получаем ID велосипеда из DataFrame
bike_id = bike_with_max_duration_df.select("bike_id").first()[0]

print(f"ID байка: {bike_id}\n Время пробега: {max_duration}")

ID байка: 535
 Время пробега: 17270400


2.   Найти наибольшее геодезическое расстояние между станциями.


In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import max, radians, sin, cos, sqrt, atan2, col, asin

# Создаем пары станций
station_pairs = stations_df.alias("st1").crossJoin(stations_df.alias("st2")).filter(col("st1.id") < col("st2.id"))

# Вычисляем расстояние между станциями по формуле Хаверсина с учётом кривизны земли
distance = sin(radians(col("st2.lat") - col("st1.lat")) / 2)**2 + cos(radians(col("st1.lat"))) * cos(radians(col("st2.lat"))) * sin(radians(col("st2.long") - col("st1.long")) / 2)**2
distance = 2 * 6371.0 * asin(sqrt(distance))

# Вычисляем максимальное расстояние между станциями
max_distance = station_pairs.select((distance).alias("distance")).agg(max("distance")).collect()[0][0]

print(f"Наибольшее геодезическое расстояние между станциями = {max_distance:.2f}")

Наибольшее геодезическое расстояние между станциями = 69.92


3.   Найти путь велосипеда с максимальным временем пробега через станции.


In [30]:
# Сортировка по столбцу duration и выбор наиболее длительной поездки
max_duration = trips_df.select("start_station_name", "end_station_name", "duration").orderBy(col("duration").desc()).first()


start_location = max_duration["start_station_name"]# Находим стартовую станцию
end_location = max_duration["end_station_name"]# Находим конечную станцию
trip_time = max_duration["duration"] # максимальное время поездки

print(f"Самый продолжительный пробег: {trip_time}секунд \nПуть был из {start_location} в {end_location}")


Самый продолжительный пробег: 17270400секунд 
Путь был из South Van Ness at Market в 2nd at Folsom


4.   Найти количество велосипедов в системе.


In [25]:
# Находим количество уникальных велосипедов
unique_bikes_count = trips_df.select("bike_id").distinct().count()
print(f"Количество велосипедов в системе: {unique_bikes_count}")

Количество велосипедов в системе: 700


5.   Найти пользователей потративших на поездки более 3 часов.

In [26]:
long_duration_trips = trips_df.filter(col("duration") > 3*60*60)# Фильтруем пользователей, отбираем потративших более 3 часов на поездки
long_duration_trips.show()# выводим результаты

+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|     start_date|  start_station_name|start_station_id|       end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+---------------+--------------------+----------------+---------------+--------------------+--------------+-------+-----------------+--------+
|4639|   11118|8/29/2013 15:18|       Market at 4th|              76|8/29/2013 18:23|       Market at 4th|            76|    433|         Customer|    null|
|4637|   11272|8/29/2013 15:17|       Market at 4th|              76|8/29/2013 18:25|       Market at 4th|            76|    377|         Customer|    null|
|4528|   12280|8/29/2013 13:39|Paseo de San Antonio|               7|8/29/2013 17:04|    Adobe on Almaden|             5|    645|       Subscriber|   94536|
|4363|   15244|8/29/2013 12:23|       Market at 4th|      

In [12]:
# Остановка SparkSession
spark.stop()